In [1]:
import subprocess
import json
import os
from constants import BTC, BTCTEST, ETH
from pprint import pprint
from getpass import getpass
from pathlib import Path
from eth_account import Account

from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3 import Account,Web3, middleware
from web3.middleware import geth_poa_middleware
w3 = Web3(Web3.HTTPProvider(os.getenv('WEB3_PROVIDER', 'http://localhost:8545')))

#w3.middleware_stack.inject(geth_poa_middleware, layer=0) # <-------- throws error
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

MNEMONIC = os.getenv("MNEMONIC")

In [2]:


def deriveWallets(coinType, numWallets, mnemonic):
    phpAndParams = f'php hd-wallet-derive-master\hd-wallet-derive.php --mnemonic="{mnemonic}" --numderive="{numWallets}" --coin="{coinType}" --format=json -g' 
    proc1 = subprocess.Popen(phpAndParams, shell=True,stdout=subprocess.PIPE)
    (output, err) = proc1.communicate()
    keyDicts = json.loads(output)
    return keyDicts



In [3]:
def create_eth_tx(account, recipient, amount):
    gasEstimate = w3.eth.estimateGas(
        {"from": account.address, "to": recipient, "value": amount}
    )
    return {
        "to": recipient,
        "from": account.address,
        "value": amount,
        "gas": gasEstimate,
        "gasPrice": w3.eth.gasPrice,
        "nonce": w3.eth.getTransactionCount(account.address)        
    }

def create_btc_tx(account, to, amount):
    return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])


def create_trx(account, to, amount,coin):
    if coin == ETH:
        return create_eth_tx(account,to,amount)
    elif coin == BTCTEST:
        return create_btc_tx(account, to, amount)


def send_tx_one(account, recipient, amount):
    tx = create_raw_tx(account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    print(result.hex())
    return result.hex()

In [4]:
ETHKeyDict = deriveWallets('eth',2,MNEMONIC)
BTCKeyDict = deriveWallets('btc-test',2,MNEMONIC)


In [5]:
print(ETHKeyDict)
print(BTCKeyDict)



[{'path': "m/44'/60'/0'/0/0", 'address': '0xEc985F1C90D4AFeD0d90f62cdE6A570E986Cf4aa', 'xprv': 'xprvA4FssuGEitX6x7Wemv7MBCcvUoKmXLLJg7ryGnLtdquLZfmpRAFf7WC7dkUxSpacGrtxQEHkgSF8XnCovanCNmkSeQHwDdWfrrSKXiDQ953', 'xpub': 'xpub6HFEHQo8ZG5QAbb7sweMYLZf2qAFvo4A3Lna5AkWCBSKSU6xxhZufJWbUzfWaFiiYhntbeNLByHgMagNDM8AspsRMzdrSRGKET2PF7zdFmV', 'privkey': '0xc7983376f390bfeddb0458c1dc0e31dd7b342ffa1b234d462b527fd99971398f', 'pubkey': '0257da1aa23732a07cbb872b681eeb1bd2059239db35600c1a71eaa26fb5350869', 'pubkeyhash': 'e8078e8c417956e3f71993042eb1852656e1fdad', 'index': 0}, {'path': "m/44'/60'/0'/0/1", 'address': '0xb340da6BEC1eBDEE435B3b4Ed052CF2261e03ae1', 'xprv': 'xprvA4FssuGEitX71wu9P22FcBVRc6r23Y3EvTdsBeLw5NqUfnNFqrsqUPYXcqMaiDZgpnY4sMA4PWG59vj8ChUnWzBgmnPmTDjcC7EGu8e9V1r', 'xpub': 'xpub6HFEHQo8ZG5QERycV3ZFyKSAA8gWSzm6HgZTz2kYdiNTYahQPQC62Bs1U7jMuN3wZeMb1n4bbNciGk6YV4imrNVJYppvmU7T1dvi8ZqUaFd', 'privkey': '0x9825ffb677bd924d9d201de41d233949444723ba2443ead6abd437bd98fe60cf', 'pubkey': '0349494db0f

In [6]:
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [7]:
ETH_account_0 = priv_key_to_account('eth', ETHKeyDict[0]['privkey'])
ETH_account_1 = priv_key_to_account('eth', ETHKeyDict[1]['privkey'])
BTC_account_0 = priv_key_to_account('btc-test', BTCKeyDict[0]['privkey'])
BTC_account_1 = priv_key_to_account('btc-test', BTCKeyDict[1]['privkey'])


print(ETH_account_0.address)
print(ETH_account_1.address)

print(BTC_account_0.address)
print(BTC_account_1.address)

0xEc985F1C90D4AFeD0d90f62cdE6A570E986Cf4aa
0xb340da6BEC1eBDEE435B3b4Ed052CF2261e03ae1
mgCnzr3MhDQBBTjmJusAGCWPdvy9Lmcc1A
n4AicaddUu2moAc9FroV3afQ4GsWRN135y


In [8]:
#send_tx('eth', ETH_account_0, ETH_account_1.address, '3')
#send_tx_1(ETH_account_0, ETH_account_1.address,1)
create_trx(ETH_account_0, ETH_account_1.address,1,ETH)


{'to': '0xb340da6BEC1eBDEE435B3b4Ed052CF2261e03ae1',
 'from': '0xEc985F1C90D4AFeD0d90f62cdE6A570E986Cf4aa',
 'value': 1,
 'gas': 21000,
 'gasPrice': 20000000000,
 'nonce': 5}

In [9]:
create_btc_tx( BTC_account_0, BTC_account_1.address, .00006)

'{"unspents":[{"amount":1962424,"confirmations":22,"script":"76a9140788e0c3fca37ac472754eef4bbe4af81c546b7a88ac","txid":"f3bf73dac1bea9a45dec0e2165cb84f6b576db9cfa58841fb107ace4eeae6cc4","txindex":1,"type":"p2pkh","vsize":148,"segwit":false},{"amount":1812196,"confirmations":574,"script":"76a9140788e0c3fca37ac472754eef4bbe4af81c546b7a88ac","txid":"0ebcabf1e6998d7e721b05660d729f9e733998a76e417f6dccd4a53c8ab28022","txindex":0,"type":"p2pkh","vsize":148,"segwit":false}],"outputs":[["n4AicaddUu2moAc9FroV3afQ4GsWRN135y",6000],["mgCnzr3MhDQBBTjmJusAGCWPdvy9Lmcc1A",3695316]]}'